In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
from os import path

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from IPython.display import SVG
import re

In [2]:
pairs = pd.read_csv('data/Main_RPAIRS_KEGG.tsv', sep='\t')

pairs['source'] = pairs['Reactant_pair'].apply(lambda x: x.split('_')[0])
pairs['target'] = pairs['Reactant_pair'].apply(lambda x: x.split('_')[1])

# Drop columns that won't be used
pairs.drop(['KEGG_reactions', 'CAR', 'RPAIR_main'], axis=1, inplace=True)
pairs.shape

(10747, 3)

In [3]:
df = pd.read_csv('data/compounds_list_KEGG.csv')
display(df.head())

print(df.shape)

,id,mol_weight,formula
0,C00002,507.1810,C10H16N5O13P3
1,C00003,664.4330,C21H28N7O14P2
2,C00005,745.4209,C21H30N7O17P3
3,C00007,31.9988,O2
4,C00011,44.0095,CO2


(5620, 3)


How many nans

In [4]:
df.isna().sum()

id              0
mol_weight    680
formula         2
dtype: int64

In [5]:
# These are probably mistakes
display(df[df['formula'].isna()])

,id,mol_weight,formula
665,C15778,NaN,NaN
2845,C20798,NaN,NaN


### Fix manually these mistakes

**Correct *C15778***

In [6]:
display(pairs[pairs['source'] == 'C15778'])

display(pairs[pairs['target'] == 'C15778'])

,Reactant_pair,source,target
787,C15778_C15781,C15778,C15781


,Reactant_pair,source,target


According to **KEGG** for the Reaction *R07492*, *C15778* should be *C15780*

In [7]:
pairs.loc[pairs.index[787], 'Reactant_pair'] = 'C15780_C15781'

df.loc[df.index[665], 'id'] = 'C15780'
df.loc[df.index[665], 'mol_weight'] = 396.6484
df.loc[df.index[665], 'formula'] = 'C28H44O'

**Correct *C20798***

In [8]:
display(pairs[pairs['source'] == 'C20798'])

display(pairs[pairs['target'] == 'C20798'])

,Reactant_pair,source,target
5112,C20798_C21180,C20798,C21180
5767,C20798_C20831,C20798,C20831


,Reactant_pair,source,target
890,C19675_C20798,C19675,C20798
2787,C11499_C20798,C11499,C20798


According to **KEGG** for the Reaction *R10719*, *C20798* should be *C21181*

In [9]:
pairs.loc[pairs.index[5112], 'Reactant_pair'] = 'C21181_C21180'
pairs.loc[pairs.index[5767], 'Reactant_pair'] = 'C21181_C20831'
pairs.loc[pairs.index[890], 'Reactant_pair'] = 'C19675_C21181'
pairs.loc[pairs.index[2787], 'Reactant_pair'] = 'C11499_C21181'

df.loc[df.index[2845], 'id'] = 'C21181'
df.loc[df.index[2845], 'mol_weight'] = 154.1417
df.loc[df.index[2845], 'formula'] = 'C3H6O5S'

Recalculate *source* and *target* in pairs

In [10]:
pairs['source'] = pairs['Reactant_pair'].apply(lambda x: x.split('_')[0])
pairs['target'] = pairs['Reactant_pair'].apply(lambda x: x.split('_')[1])

pairs.to_csv('data/Main_RPAIRS_KEGG_fixed.csv', index=None)

## Exctract Features From Chemical Formula

In [11]:
def extract_elements(df, column_name):
    # define the regular expression pattern to match the chemical formula
    pattern = r'[A-Z][a-z]?'
    # initialize a set to store the element symbols
    elements = set()
    # loop over the values in the specified column of the DataFrame
    for value in df[column_name].values:
        # find all matches of the pattern in the value string
        matches = re.findall(pattern, value)
        # add the matches to the set of elements
        elements.update(matches)
    return elements

def extract_stoichiometry(formula):
    # define the regular expression pattern to match the chemical formula
    pattern = r'([A-Z][a-z]?)(\d*)'
    # initialize the dictionary to store the element symbol and its stoichiometry
    stoichiometry = {}
    # loop over the matches of the pattern in the formula string
    for match in re.findall(pattern, formula):
        symbol, count = match
        # if the count is empty, set it to 1
        count = int(count) if count else 1
        # add the symbol and count to the stoichiometry dictionary
        stoichiometry[symbol] = count
    return stoichiometry

# example usage
elements = extract_elements(df, 'formula')
print(elements)

# Create a col for every element
for elm in elements: df[elm]=0

{'Cl', 'P', 'Zn', 'Se', 'N', 'O', 'Br', 'Mg', 'As', 'R', 'X', 'C', 'Co', 'Mo', 'F', 'H', 'S', 'I'}


In [12]:
for row in range(len(df)):
    
    formula = df['formula'].iloc[row]
    stoichiometry = extract_stoichiometry(formula)
    
    for key, value in stoichiometry.items():
        #df[key].iloc[row] = value
        df.loc[df.index[row], key] = value

display(df.head(3))

,id,mol_weight,formula,Cl,P,Zn,Se,N,O,Br,...,As,R,X,C,Co,Mo,F,H,S,I
0,C00002,507.1810,C10H16N5O13P3,0,3,0,0,5,13,0,...,0,0,0,10,0,0,0,16,0,0
1,C00003,664.4330,C21H28N7O14P2,0,2,0,0,7,14,0,...,0,0,0,21,0,0,0,28,0,0
2,C00005,745.4209,C21H30N7O17P3,0,3,0,0,7,17,0,...,0,0,0,21,0,0,0,30,0,0


In [13]:
# Many many compounds with R
print(len(df[df['R']!=0]))

# Col that contains the info if the compound is a polymer or not
df['polymer'] = 0

for row in range(len(df)):
    if 'n' in df['formula'].iloc[row]: 
        df.loc[df.index[row], 'polymer'] = 1
        
# Print how many polymers do we have
print(len(df[df['polymer'] == 1]))

# save to csv
df.to_csv('data/curated.csv')

597
129


**Here we re-fill all mol_weight based on paper NICEpath methodology.**

# Load curated dataset from Stef - Filled NaN `mol_weight`

In [14]:
df = pd.read_csv('data/curated_complete_mw.csv', index_col=0)
df.drop('mol_weight', axis=1, inplace=True)
df.drop('R.1', axis=1, inplace=True)
df.drop('Mr', axis=1, inplace=True)
df.rename({'N.1':'mol_weight'}, axis=1, inplace=True)
df.head()

,id,formula,Co,C,Br,Zn,N,Mo,S,Se,...,X,H,O,As,F,R,P,Cl,polymer,mol_weight
0,C00002,C10H16N5O13P3,0,10,0,0,5,0,0,0,...,0,16,13,0,0,0,3,0,0,507.0
1,C00003,C21H28N7O14P2,0,21,0,0,7,0,0,0,...,0,28,14,0,0,0,2,0,0,664.0
2,C00005,C21H30N7O17P3,0,21,0,0,7,0,0,0,...,0,30,17,0,0,0,3,0,0,745.0
3,C00007,O2,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,32.0
4,C00011,CO2,0,1,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,44.0


In [15]:
source_mw = df.set_index('id')['mol_weight']
target_mw = source_mw.reindex(pairs['target']).values

pairs['MW'] = abs(source_mw.reindex(pairs['source']).values - target_mw) / (source_mw.reindex(pairs['source']).values + target_mw)

display(pairs.head(3))

,Reactant_pair,source,target,MW
0,C00002_C07024,C00002,C07024,0.001976
1,C00003_C00004,C00003,C00004,0.000752
2,C00005_C00006,C00005,C00006,0.000672


# Import to Networkx 

In [16]:
import networkx as nx

G = nx.Graph()

# iterate over the rows of the DataFrame and add each edge to the graph
for index, row in pairs.iterrows():
    source = row['source']
    target = row['target']
    G.add_edge(source, target)

# Add df cols as node features
node_data = df.set_index('id').to_dict('index')

# Add the node features to the graph
for node, data in G.nodes(data=True):
    node_features = node_data.get(node)
    if node_features:
        data.update(node_features)

print('# nodes:', G.number_of_nodes(), "\n# edges:", G.number_of_edges())

# nodes: 5620 
# edges: 10747


In [17]:
if path.exists('data/nodes_centralities.csv'):
    dc = pd.read_csv('data/nodes_centralities.csv', index_col=0)
else:
    # PageRank
    pr = nx.pagerank(G)
    pr = pd.DataFrame(list(pr.items()), columns=['Node', 'PageRank'])

    # degree centrality
    dc = nx.degree_centrality(G)
    dc = pd.DataFrame(list(dc.items()), columns=['Node', 'Degree Centrality'])
    
    # Betweenness centrality
    bc = nx.betweenness_centrality(G)
    bv = pd.DataFrame(list(bc.items()), columns=['Node', 'Betweenness Centrality'])
    
    # centralities
    dc['PageRank'] = pr['PageRank'].copy()
    dc['Betweenness Centrality'] = bc['Betweenness Centrality'].copy()
    dc.sort_values(by='PageRank', ascending=False, inplace=True)
    dc = pd.merge(dc, df[['id','formula','mol_weight']].rename({'id':'Node'}, axis=1), on='Node')
    dc.to_csv('data/nodes_centralities.csv')

#### Scale the centrality measures

In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dc[['Degree Centrality', 'PageRank', 'Betweenness Centrality']] = scaler.fit_transform(dc[['Degree Centrality', 'PageRank', 'Betweenness Centrality']])

print('Centrality measures statistics: ')
display(dc.describe().T)

Centrality measures statistics: 


,count,mean,std,min,25%,50%,75%,max
Degree Centrality,5620.0,0.003087,0.019770,0.0000,1.092896e-03,0.001093,0.003279,1.0000
PageRank,5620.0,0.003621,0.020670,0.0000,1.266360e-03,0.002221,0.003468,1.0000
Betweenness Centrality,5620.0,0.001034,0.016605,0.0000,1.114510e-07,0.000079,0.000719,1.0000
mol_weight,5620.0,440.817497,328.063873,17.0305,1.881794e+02,320.466300,589.540650,2484.8138


In [19]:
# Number of compounds with centrality measure over the 75% threshold
print(len(dc[dc['Betweenness Centrality'] > 0.000719]))

print(len(dc[dc['Degree Centrality'] > 0.0034680]))

print(len(dc[dc['PageRank'] > 0.003279]))

1055
785
1572


In [20]:
# top_centrality_nodes = list(dc[dc['Degree Centrality'] > 0.0034680]['Node'].values)

# subset_df = df[df['id'].isin(top_centrality_nodes)]
# subset_df

In [21]:
# Concat to df the centrality measures from dc
df = pd.merge(df, dc[['Node', 'PageRank', 'Degree Centrality', 'Betweenness Centrality']], left_on='id', right_on='Node')
df.drop('Node', axis=1, inplace=True)
df.head(2)

,id,formula,Co,C,Br,Zn,N,Mo,S,Se,...,As,F,R,P,Cl,polymer,mol_weight,PageRank,Degree Centrality,Betweenness Centrality
0,C00002,C10H16N5O13P3,0,10,0,0,5,0,0,0,...,0,0,0,3,0,0,507.0,0.220897,0.244809,0.147827
1,C00003,C21H28N7O14P2,0,21,0,0,7,0,0,0,...,0,0,0,2,0,0,664.0,0.007615,0.009836,0.000216


In [22]:
# Create new mw adding info from centrality measure
df['mw'] = df['mol_weight'] + df['mol_weight'] * df['PageRank']

# Update the edge weight
source_mw = df.set_index('id')['mw']
target_mw = source_mw.reindex(pairs['target']).values

pairs['MW_wCentr'] = abs(source_mw.reindex(pairs['source']).values - target_mw) / (source_mw.reindex(pairs['source']).values + target_mw)

display(pairs.head(3))

,Reactant_pair,source,target,MW,MW_wCentr
0,C00002_C07024,C00002,C07024,0.001976,0.100431
1,C00003_C00004,C00003,C00004,0.000752,0.001602
2,C00005_C00006,C00005,C00006,0.000672,0.000305


# Node2Vec

In [23]:
from karateclub import Node2Vec

# Map node names to integers
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)

" Perform node embedding using Node2Vec "
N2vec_model = Node2Vec(walk_number=3, walk_length=20,q=.25,\
                       dimensions=124)
N2vec_model.fit(G)
N2Vec_embedding = N2vec_model.get_embedding()
print('Embedding array shape (nodes x features):',N2Vec_embedding.shape )

KeyboardInterrupt: 

In [ ]:
# Add a new column to data.x with the node names from the mapping
node_names = [None] * len(N2Vec_embedding)

for node, index in mapping.items():
    node_names[index] = node   
n2v_data = np.column_stack((N2Vec_embedding, node_names))
n2v_data = pd.DataFrame(n2v_data)

In [ ]:
n2v_data

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# Suppose you have the node embeddings in a dictionary called 'embeddings'
u_embedding = n2v_data.iloc[1666,:-1]
v_embedding = n2v_data.iloc[3607,:-1]

similarity = cosine_similarity(u_embedding.values.reshape(1, -1), \
                               v_embedding.values.reshape(1, -1))[0, 0]
print(similarity)

In [ ]:
n2v_data[n2v_data[124] == 'C0022']

In [ ]:
n2v_data[n2v_data[124] == 'C12096']

In [ ]:
n2v_data[n2v_data[124] == 'C00323']

## Add edge attributes

In [ ]:
def get_weights(a,b, method):
    
    # a_C and b_C is the number of C at each compound
    '''
    Checking if we go from a compound with C to a compound with no C
    '''
    a_C = df[df['id']==a]['C'].values
    b_C = df[df['id']==b]['C'].values
    
    if (a_C != 0 and b_C == 0) or (a_C == 0 and b_C != 0):
        return 999
    
    try:
        w = pairs[(pairs['source'] == a) & (pairs['target'] == b)][method].values[0]
    except IndexError:
        w = pairs[(pairs['source'] == b) & (pairs['target'] == a)][method].values[0]
    
    return w

for edge in G.edges():
    G.edges[(edge[0], edge[1])]['weight'] = \
            get_weights(edge[0], edge[1], method='MW')

In [ ]:
source = 'C00082'
target = 'C01533'
list(nx.shortest_path(G, source, target, weight='weight'))

### Load file with test cases fron NICEpath

In [ ]:
test_cases = pd.read_csv('data/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases.head(3)

In [ ]:
for row in range(len(test_cases)):
    source = test_cases['source'].iloc[row]
    target = test_cases['target'].iloc[row]
    print(list(nx.shortest_path(G, source, target, weight='weight')))

### Get node embeddings using GNN and Clustering (idea)

In [ ]:
# Backup df
df_copy = df.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

'''Scale df'''
# Select the columns with non-object data types
numeric_cols = df.select_dtypes(include=['int', 'float']).columns

# Apply standard scaling to the selected columns
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

'''Recreate Graph'''
G = nx.Graph()

# iterate over the rows of the DataFrame and add each edge to the graph
for index, row in pairs.iterrows():
    source = row['source']
    target = row['target']
    G.add_edge(source, target)

# Add df cols as node features
node_data = df.set_index('id').to_dict('index')

# Add the node features to the graph
for node, data in G.nodes(data=True):
    node_features = node_data.get(node)
    if node_features:
        data.update(node_features)

print('# nodes:', G.number_of_nodes(), "\n# edges:", G.number_of_edges())

In [ ]:
from torch_geometric.utils.convert import from_networkx
import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv
import torch
import torch.nn as nn
import torch.nn.functional as F

node_features = [col for col in df.drop(['id', 'formula'], axis=1).columns]

data = from_networkx(G, group_node_attrs=node_features)
print(data.x.shape)

# Map node names to integers
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)

# Add a new column to data.x with the node names from the mapping
node_names = [None] * len(data.x)
for node, index in mapping.items():
    node_names[index] = node
    
graph_data = np.column_stack((data.x, node_names))
graph_data = pd.DataFrame(graph_data)

In [ ]:
'''
Graph embedding good method (maybe)
'''
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GAE

# Define the model architecture
class Encoder(torch.nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(data.num_node_features, 4)
        self.conv2 = GCNConv(4, 8)
        self.conv3 = GCNConv(8, 4)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        X = F.relu(x)
        x = self.conv3(x, edge_index)
        return x

class Decoder(torch.nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.linear = torch.nn.Linear(4, data.num_node_features)

    def forward(self, z):
        x = self.linear(z)
        return torch.sigmoid(x)

model = GAE(Encoder(), Decoder())

# Define the loss function and the optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z)
    loss = criterion(out, data.x)
    loss.backward()
    optimizer.step()
    
    if(epoch % 20 == 0):
        print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, loss.item()))
    
    
model.eval()
emb = model.encode(data.x, data.edge_index)
print(emb.shape)

In [ ]:
node_embeddings=pd.DataFrame(emb.detach().numpy())
node_embeddings['id']  = graph_data[graph_data.shape[1]-1]
# node_embeddings.set_index('id', inplace=True)
node_embeddings

In [ ]:
display(node_embeddings[node_embeddings['id'] == 'C00031'])
display(node_embeddings[node_embeddings['id'] == 'C00002'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# Suppose you have the node embeddings in a dictionary called 'embeddings'
u_embedding = node_embeddings.iloc[23,:-1]
v_embedding = node_embeddings.iloc[5619,:-1]

similarity = cosine_similarity(u_embedding.values.reshape(1, -1), v_embedding.values.reshape(1, -1))[0, 0]
print(similarity)

In [ ]:
class SAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers):
        super(SAGE, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            self.convs.append(SAGEConv(in_channels, hidden_channels))

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x

    def full_forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SAGE(data.num_node_features, hidden_channels=4, \
             num_layers=1)

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

x, edge_index = data.x.to(device), data.edge_index.to(device)

# Forward propagate the graph through the model
model.eval()
with torch.no_grad():
    out = model.full_forward(data.x, data.edge_index)

# Extract the node embeddings
node_embeddings = out.numpy()  # or out.detach().numpy() if using PyTorch version <1.6

# Print the shape of the node embeddings matrix
print('Node embeddings shape:', node_embeddings.shape)

In [ ]:
node_embeddings=pd.DataFrame(node_embeddings)
node_embeddings['id']  = graph_data[graph_data.shape[1]-1]
# node_embeddings.set_index('id', inplace=True)
node_embeddings

In [ ]:
node_embeddings[node_embeddings['id'] == 'C00811']

In [ ]:
node_embeddings[node_embeddings['id'] == 'C00013']

In [ ]:
node_embeddings[node_embeddings['id'] == 'C00146']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# Suppose you have the node embeddings in a dictionary called 'embeddings'
u_embedding = node_embeddings.iloc[62,:-1]
v_embedding = node_embeddings.iloc[3809,:-1]

similarity = cosine_similarity(u_embedding.values.reshape(1, -1), v_embedding.values.reshape(1, -1))[0, 0]
print(similarity)

In [ ]:
similarity, _ = pearsonr(u_embedding, v_embedding)
print(similarity)

In [ ]:
distance = np.linalg.norm(u_embedding - v_embedding)
similarity = 1 / (1 + distance)
print(similarity)

In [ ]:
# Add cosine similarity from embedding to pairs 
cos_sim = []
for row in range(len(pairs)):
    
    source = pairs['source'].iloc[row]
    target = pairs['target'].iloc[row]
    
    u_embedding = node_embeddings[node_embeddings['id'] == source].iloc[0, :-1]
    v_embedding = node_embeddings[node_embeddings['id'] == target].iloc[0, :-1]

    similarity = cosine_similarity(u_embedding.values.reshape(1, -1), v_embedding.values.reshape(1, -1))[0, 0]
    cos_sim.append(similarity)
    
pairs['cos_sim'] = cos_sim
# pairs['cos_sim'] = (pairs['cos_sim'] + 1) / 2
pairs['cos_sim'] = 1 - pairs['cos_sim']

pairs.head()

In [ ]:
print(pairs['cos_sim'].describe())

pairs['cos_sim'] = pairs['cos_sim'].apply(lambda x: max(0,x))

# Import to Networkx 

In [ ]:
import networkx as nx

df = df_copy.copy()
G = nx.Graph()

# iterate over the rows of the DataFrame and add each edge to the graph
for index, row in pairs.iterrows():
    source = row['source']
    target = row['target']
    G.add_edge(source, target)

# Add df cols as node features
node_data = df.set_index('id').to_dict('index')

# Add the node features to the graph
for node, data in G.nodes(data=True):
    node_features = node_data.get(node)
    if node_features:
        data.update(node_features)

print('# nodes:', G.number_of_nodes(), "\n# edges:", G.number_of_edges())

## Add edge attributes

In [ ]:
def get_weights(a,b, method):
    
    # a_C and b_C is the number of C at each compound
    a_C = df[df['id']==a]['C'].values
    b_C = df[df['id']==b]['C'].values
    if (a_C != 0 and b_C == 0) or (a_C == 0 and b_C != 0):
        return 999
    
    try:
        w = pairs[(pairs['source'] == a) & (pairs['target'] == b)][method].values[0]
    except IndexError:
        w = pairs[(pairs['source'] == b) & (pairs['target'] == a)][method].values[0]
    
    return w

for edge in G.edges():
    G.edges[(edge[0], edge[1])]['weight'] = \
            get_weights(edge[0], edge[1], method='cos_sim')

In [ ]:
source = 'C00082'
target = 'C01533'
list(nx.shortest_path(G, source, target, weight='weight'))

In [ ]:
source = 'C07481'
target = 'C00385'
list(nx.shortest_path(G, source, target, weight='weight'))

### Load file with test cases fron NICEpath

In [ ]:
test_cases = pd.read_csv('data/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases.head(3)

In [ ]:
for row in range(len(test_cases)):
    source = test_cases['source'].iloc[row]
    target = test_cases['target'].iloc[row]
    print(list(nx.shortest_path(G, source, target, weight='weight')))